# This notebook is part 1 of using langchain with local ai models to query an arbitrary database.
In this notebook I will try to use langchain with local models to generate a SQL query, run the query, then format the results into natural language.  I will be using the other unnumbered langchain notebooks as a reference.  I'll also be using some of the logic from the previous autogen notebooks.

In [1]:
%pip install python-dotenv
%pip install langchain
%pip install pip install pyodbc

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
# connect to MSSQL database
import pyodbc

SERVER = '127.0.0.1'
DATABASE = 'TimeBasedCommitments'
USERNAME = 'sa'
PASSWORD = 'BadDefaultPassword!'

connectionString = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={SERVER};DATABASE={DATABASE};UID={USERNAME};PWD={PASSWORD}'

conn = pyodbc.connect(connectionString)

# conn.close()

In [7]:
# Get table names
get_all_tables_stmt = "SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES;"
cursor = conn.cursor()
cursor.execute(get_all_tables_stmt)
table_names = [row[0] for row in cursor.fetchall()]

# Get all table definitions and format them into CREATE TABLE statements
get_def_stmt = f"""
SELECT COLUMN_NAME, DATA_TYPE
FROM INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_NAME = ?
"""

definitions = []
for table_name in table_names:
    cursor.execute(get_def_stmt, (table_name,))
    rows = cursor.fetchall()

    create_table_stmt = f"CREATE TABLE {table_name} (\n"
    for row in rows:
        create_table_stmt += f"{row[0]} {row[1]},\n"
    create_table_stmt = create_table_stmt.rstrip(",\n") + "\n);"
    
    definitions.append(create_table_stmt)

table_definitions = "\n\n".join(definitions)

In [4]:
import os
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.
KOBOLD_API_BASE = os.getenv('KOBOLD_API_BASE')
print(KOBOLD_API_BASE)

http://localhost:5001/api/


In [35]:
from langchain.llms import KoboldApiLLM
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = KoboldApiLLM(endpoint=KOBOLD_API_BASE, temperature=0.1, )

template = """### Instruction:
Create a SQL query that can be used to answer this question: {question}

Use these TABLE_DEFINITIONS to satisfy the database query.

TABLE_DEFINITIONS

{table_definitions}

Respond only with the SQL query as raw text. I need to be able to easily parse the sql query from your response.
Do not use the \ character.

### Response:
"""

prompt = PromptTemplate(template=template, input_variables=["question", "table_definitions"])

llm_chain = LLMChain(prompt=prompt, llm=llm)

question = "Who completed the most specific commitments?"

sql_query = llm_chain.run(question = question, table_definitions = table_definitions)

print(sql_query)

<>:7: SyntaxWarning: invalid escape sequence '\ '
<>:7: SyntaxWarning: invalid escape sequence '\ '
C:\Users\kgage\AppData\Local\Temp\ipykernel_18728\1968013939.py:7: SyntaxWarning: invalid escape sequence '\ '
  template = """### Instruction:


SELECT TOP 1 completed_by, COUNT(*) AS CompletedCount
FROM tblSpecificCommitments
WHERE completed IS NOT NULL
GROUP BY completed_by
ORDER BY CompletedCount DESC


In [36]:
cursor = conn.cursor()
cursor.execute(sql_query)
rows = cursor.fetchall()
print(rows)

[(1, 1)]


In [37]:
template = """### Instruction:
Answer this question: {question}

Assume this SQL query was run: {sql_query}

Assume this was the data returned from the query: {data}

### Response:
"""

prompt = PromptTemplate(template=template, input_variables=["question", "sql_query", "data"])

llm_chain = LLMChain(prompt=prompt, llm=llm)

response = llm_chain.run(question = question, sql_query = sql_query, data = rows)

print(response)

The person who completed the most specific commitments is identified by the ID '1' and they have completed a total of 1 commitment.


In [38]:
conn.close()